In [1]:
import sys
sys.path.append('/home/lo276838/Modèles/mri-dl-recon/src/mri_dlrecon')

In [2]:
from fastmri.data import transforms as T
import torch
import numpy as np
from matplotlib import pyplot as plt
from torch.autograd import Variable

from utils_torch import virtual_coil_reconstruction, load_and_transform, create_zero_filled_reconstruction
from fastmri.data.subsample import create_mask_for_mask_type
from fastmri.models.varnet import VarNet 


In [3]:
plt.rcParams['figure.figsize'] = (7, 7)
# plt.rcParams['image.cmap'] = 'gray'

In [4]:
file_path = "/volatile/FastMRI/brain_multicoil_train/multicoil_train/file_brain_AXT1POST_201_6002780.h5"

kspace_multicoil = load_and_transform(file_path)
images_multicoil = torch.fft.fftshift(torch.fft.ifft2(kspace_multicoil))

image = virtual_coil_reconstruction(images_multicoil)
print(image.shape)
kspace = torch.fft.fft2(image)

mask_type = "random"
center_fractions = [0.08, 0.04]
accelerations = [4, 8]
mask = create_mask_for_mask_type(mask_type, center_fractions, accelerations)

zero_filled_reconstruction , test = create_zero_filled_reconstruction(mask,kspace)
zero_filled_reconstruction = torch.fft.fftshift(zero_filled_reconstruction)
print(test.shape)


input = zero_filled_reconstruction
target = image.abs().unsqueeze(1)

torch.Size([16, 640, 320])
torch.Size([1, 640, 1])


In [5]:
# plt.subplot(1,2,1)
# plt.imshow(np.abs(input[0, 0, :,: ].numpy()))
# plt.title("Input")

# plt.subplot(1,2,2)
# plt.imshow(target[0,0, :,: ].numpy())
# plt.title("Target")

# plt.tight_layout()
# plt.show()

In [6]:
chans = 32
num_pool_layers = 4
lr = 0.0001

model = VarNet(chans=chans, pools=num_pool_layers)
criterion = torch.nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

loss_list = []
print(test.shape)
num_epochs = 1
for epoch in range(num_epochs):
    input = Variable(input)
    target = Variable(target)
    
    test = test.unsqueeze(1)
    test = test.unsqueeze(1)
    input = input.unsqueeze(-1)
    print(input.shape)
    print(test.shape)
    input.shape[-1] = 2
    print(input.shape[-1])

    optimizer.zero_grad()
    outputs = model(input, test)
    loss = criterion(outputs, target)
    loss_list.append(loss.item())

    loss.backward()
    optimizer.step()

    # print(f'Epoch [{epoch+1}/{num_epochs}]')

# Sauvegarder le modèle
torch.save(model.state_dict(), 'fastmri_unet_model.pth')


torch.Size([1, 640, 1])
torch.Size([16, 1, 640, 320, 1])
torch.Size([1, 1, 1, 640, 1])


TypeError: 'torch.Size' object does not support item assignment